In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import numpy as np
import seaborn as sns
import random
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
import cv2
from sklearn import metrics
import newSom
from mayavi import mlab

In [2]:
def stringToInt(X,name):
    X[name] = X[name].astype(str).str.strip()
    #print(X[name])
    for i in range(0, X.shape[0]):
        if  X.at[i,name]== 'Jan':
            X.at[i,name] =1            
        elif  X.at[i,name]== 'Feb':
                X.at[i,name]=2 
        elif  X.at[i,name]== 'Mar': 
            X.at[i,name]=3 
        elif  X.at[i,name]== 'Apr': 
            X.at[i,name]=4 
        elif  X.at[i,name]== 'May': 
            X.at[i,name]=5 
        elif  X.at[i,name]== 'June': 
            X.at[i,name]=6  
        elif  X.at[i,name]== 'Jul': 
            X.at[i,name]=7 
        elif  X.at[i,name]== 'Aug':
            X.at[i,name]=8 
        elif  X.at[i,name]== 'Sep':
            X.at[i,name]=9 
        elif  X.at[i,name]== 'Oct':
            X.at[i,name]=10
        elif  X.at[i,name]== 'Nov':
            X.at[i,name]=11 
        elif  X.at[i,name]== 'Dec': 
            X.at[i,name]=12
        elif  X.at[i,name]== 'Returning_Visitor':  
            X.at[i,name]=0
        elif  X.at[i,name]== 'New_Visitor': 
            X.at[i,name]=1
        elif  X.at[i,name]== 'Other': 
            X.at[i,name]=3
        elif  X.at[i,name]== 'False': 
            X.at[i,name]=0        
        elif  X.at[i,name]== 'True': 
            #print(X.at[i,name])
            X.at[i,name]=1
            #print(X.at[i,name])
        else: 
            print("Unkonwn value {}".format(X.at[i,name]))


In [27]:
csv = pd.read_csv("online_shoppers_intention.csv")
del csv["Month"]
del csv["Weekend"]
stringToInt(csv,"VisitorType")
stringToInt(csv,"Revenue")
#sample number of train and test dataset
train_num = 5000
test_num = 5000
#get samples
data_train = csv.sample(train_num)

data_test = csv.drop(data_train.index)
print(data_test.shape)
print(data_test.index)
#data_train.to_numpy(dtype=np.float64)
#data_test = csv.sample(test_num).to_numpy(dtype=np.float64)
#
#
#label_train = data_train[:,data_train.shape[1]-1]
#label_test = data_test[:,data_test.shape[1]-1]
## delete the last column
#data_train= data_train[:,:-1]
#data_test= data_test[:,:-1]
#print(label_test)


(7330, 16)
Int64Index([    0,     2,     5,     6,     7,     8,     9,    10,    13,
               15,
            ...
            12313, 12314, 12315, 12317, 12318, 12324, 12325, 12326, 12327,
            12329],
           dtype='int64', length=7330)


In [4]:
def NormalizeLables(X,Y,category = 0):
    """
    transfrer predcited label to match the range of true label
    a = np.amax(X)+1,b = np.amax(Y)+1 the number of classes in X and Y
    b= m*n,  to make it easier , to make b can be divided by a
    category = 0 noramlize label
    category = 1 normalize sub label
    """
    div = int((np.amax(Y)+1)/(np.amax(X)+1))
    #print("div : {}".format(div) )
    if(category == 0):
        global nLabel 
        nLabel = np.arange(Y.size)
    if(category == 1):
        global nsubLabel 
        nsubLabel = np.arange(Y.size)
    index = 0;
    for idx, y in enumerate(Y): 
       # print("idx {}".format(idx))
        for i in range(1,div+1):
            if(y < i*div):
                index+=1
                #print(i-1)
                if(category == 0):
                    nLabel[idx] = i-1
                if(category == 1):
                    nsubLabel[idx] = i-1
                break               
    if(category == 0):
        print("normalized predicted nLabel:\n {} ".format(nLabel))
    if(category == 1):
        print("normalized predicted nsubLabel: \n{} ".format(nsubLabel))
    return 

In [5]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    #print(y_true.shape)
    #print(y_pred.shape)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
   # print (np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix))
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

In [6]:
def groupClusterList(m,Y):
    """
    transfer all label in a list
    m: cluster Number
    """
    clusters = []
    for i in range(0,m):
        newlist = []
        for idx, y in enumerate(Y): 
            if(y == i):
                newlist.append(idx)
        clusters.append(newlist) 
   # print(clusters)
    return clusters

In [7]:
def getNoisyClusters(list_true,list_pred):
    """
    get the wrong data indices in the training data when predicted with weight1
    """
    noisylist = []
    for i in range(0,len(list_true)):
        newlist = [item for item in list_pred[i] if item not in list_true[i]]
        noisylist.append(newlist)
    print("noisy data:")
    return noisylist

In [8]:
def reduce_noisy_data(noisy_list, percent):
    newlist = []
    for x in noisy_list:
        for e in x:
            newlist.append(e)
   # print(newlist)
   # print(len(newlist))
   # print(int(percent * len(newlist)))
    newlist = random.sample(newlist, int(percent * len(newlist)))
   # print(newlist)
    return newlist

In [9]:
def reduce_data_byindice(reduced_indices,X,category = 0):
   
    if(category == 0): 
        global data_train_subset
        data_train_subset= np.delete(X, reduced_indices, axis=0)
        #print("reduced_indices :{}".format(reduced_indices))
    if(category == 1): 
        global data_train_sublabel
        data_train_sublabel=np.delete(X,reduced_indices, axis=0)
    #print(df_subset.to_numpy())   
    return 

In [10]:

som1 = newSom.SOM(m=4, n=4, dim=15)

In [11]:
som1.fit(data_train)

generate weight1:
 [6.09199190e-08 1.67529777e-07 0.00000000e+00 0.00000000e+00
 4.00000061e+00 1.07000031e+02 4.12018774e-10 5.00000000e-02
 0.00000000e+00 5.99999991e-01 2.99999997e+00 1.99999998e+00
 2.99999997e+00 1.29999998e+01 0.00000000e+00]


In [12]:
predictions1_w1 = som1.predict(data_train,som1.weights1)

weights used:

[6.09199190e-08 1.67529777e-07 0.00000000e+00 0.00000000e+00
 4.00000061e+00 1.07000031e+02 4.12018774e-10 5.00000000e-02
 0.00000000e+00 5.99999991e-01 2.99999997e+00 1.99999998e+00
 2.99999997e+00 1.29999998e+01 0.00000000e+00]
predicted label:
 [15 15  0 ... 15 15 11]


In [13]:
NormalizeLables(label_train,predictions1_w1)

normalized predicted nLabel:
 [1 1 0 ... 1 1 1] 


In [14]:
purity_score(label_train,nLabel)

0.8352

In [15]:
global noisy_list
noisy_list = getNoisyClusters(groupClusterList(2,label_train),groupClusterList(2,nLabel))
reduced_indices = reduce_noisy_data(noisy_list,0.1)
reduce_data_byindice(reduced_indices,data_train)
reduce_data_byindice(reduced_indices,label_train,1)

noisy data:


In [16]:
som1.fit(data_train_subset,weightIndex =1)

generate weight2:
 [1.29320341e+00 7.61249965e+01 7.10222973e-02 3.55427042e+00
 2.42463730e+01 8.33853972e+02 1.11796334e-02 4.10507765e-02
 2.62480752e+00 2.36557765e-03 2.88220061e+00 2.04288808e+00
 2.42665001e+00 1.92505096e+00 2.44439154e-02]


In [17]:
predictions2_W2 = som1.predict(data_train_subset,som1.weights2)
NormalizeLables(data_train_sublabel,predictions2_W2,1)
purity_score(data_train_sublabel,nsubLabel)

weights used:

[1.29320341e+00 7.61249965e+01 7.10222973e-02 3.55427042e+00
 2.42463730e+01 8.33853972e+02 1.11796334e-02 4.10507765e-02
 2.62480752e+00 2.36557765e-03 2.88220061e+00 2.04288808e+00
 2.42665001e+00 1.92505096e+00 2.44439154e-02]
predicted label:
 [ 3  6 12 ...  2  3  1]
normalized predicted nsubLabel: 
[0 0 1 ... 0 0 0] 


0.8386161176714315

In [18]:
predictions_test_W1 = som1.predict(data_test,som1.weights1)
NormalizeLables(label_test,predictions_test_W1,0)
purity_score(label_test,nLabel)

weights used:

[6.09199190e-08 1.67529777e-07 0.00000000e+00 0.00000000e+00
 4.00000061e+00 1.07000031e+02 4.12018774e-10 5.00000000e-02
 0.00000000e+00 5.99999991e-01 2.99999997e+00 1.99999998e+00
 2.99999997e+00 1.29999998e+01 0.00000000e+00]
predicted label:
 [ 0  6 15 ...  4  9  0]
normalized predicted nLabel:
 [0 0 1 ... 0 1 0] 


0.8466

In [19]:
predictions_test_W2 = som1.predict(data_test,som1.weights2)
NormalizeLables(label_test,predictions_test_W2,0)
purity_score(label_test,nLabel)

weights used:

[1.29320341e+00 7.61249965e+01 7.10222973e-02 3.55427042e+00
 2.42463730e+01 8.33853972e+02 1.11796334e-02 4.10507765e-02
 2.62480752e+00 2.36557765e-03 2.88220061e+00 2.04288808e+00
 2.42665001e+00 1.92505096e+00 2.44439154e-02]
predicted label:
 [12 10  3 ... 12  4 12]
normalized predicted nLabel:
 [1 1 0 ... 1 0 1] 


0.8466